# 블록체인프로그래밍 12주
## (1) REMIX에서 주문하고, 이벤트 발생하는지 보이도록 콘솔 출력 붙여넣으세요.
### 코드

In [39]:
%%writefile src/OrderEvent.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.8.0;

contract OrderEvent {
    address owner;

    event OrderLog(address indexed _from, bytes2 _itemId, uint _quantity, uint _price, string _shipAddress);

    constructor() {
        owner = msg.sender;
    }
    function order(bytes2 _itemId, uint quantity, uint unitPrice, string memory shipAddress) public payable {
        uint256 orderAmount = quantity * unitPrice;
        require(msg.value == orderAmount);
        emit OrderLog(msg.sender, _itemId, quantity, unitPrice, shipAddress);
    }
    function getBalance() public view isOwner returns(uint) {
        return address(this).balance;
    }
    modifier isOwner {
        require(msg.sender == owner);
        _;
    }
}

Overwriting src/OrderEvent.sol


### 로그
#### 배포
```
creation of OrderEvent pending...
[vm]from: 0x5B3...eddC4to: OrderEvent.(constructor)value: 0 weidata: 0x608...70033logs: 0hash: 0x65b...47138
status	true Transaction mined and execution succeed
transaction hash	0x65b61243128757784681a7264d65017898c6a458fac931ab98313af3df647138
from	0x5B38Da6a701c568545dCfcB03FcB875f56beddC4
to	OrderEvent.(constructor)
gas	437196 gas
transaction cost	380170 gas 
execution cost	380170 gas 
input	0x608...70033
decoded input	{}
decoded output	 - 
logs	[]
val	0 wei
```
#### order
```
transact to OrderEvent.order pending ... 
[vm]from: 0x5B3...eddC4to: OrderEvent.order(bytes2,uint256,uint256,string) 0xcD6...99Df9value: 300 weidata: 0x5a2...00000logs: 1hash: 0xcd1...8411e
status	true Transaction mined and execution succeed
transaction hash	0xcd1a3f90211b2279deeb85393b291cf713fb4dc6df9bd300a8a06c56bde8411e
from	0x5B38Da6a701c568545dCfcB03FcB875f56beddC4
to	OrderEvent.order(bytes2,uint256,uint256,string) 0xcD6a42782d230D7c13A74ddec5dD140e55499Df9
gas	32235 gas
transaction cost	28030 gas 
execution cost	28030 gas 
input	0x5a2...00000
decoded input	{
	"bytes2 _itemId": "0x1111",
	"uint256 quantity": "3",
	"uint256 unitPrice": "100",
	"string shipAddress": "20 2-gil Hongji-dong Jongro-gu Seoul"
}
decoded output	{}
logs	[
	{
		"from": "0xcD6a42782d230D7c13A74ddec5dD140e55499Df9",
		"topic": "0x538006ae16981d84b66e084c148f73553be69e4290d2c7bbb61abec6284186c1",
		"event": "OrderLog",
		"args": {
			"0": "0x5B38Da6a701c568545dCfcB03FcB875f56beddC4",
			"1": "0x1111",
			"2": "3",
			"3": "100",
			"4": "20 2-gil Hongji-dong Jongro-gu Seoul",
			"_from": "0x5B38Da6a701c568545dCfcB03FcB875f56beddC4",
			"_itemId": "0x1111",
			"_quantity": "3",
			"_price": "100",
			"_shipAddress": "20 2-gil Hongji-dong Jongro-gu Seoul"
		}
	}
]
val	300 wei
```
#### getBalance
```
call to OrderEvent.getBalance
CALL
[call]from: 0x5B38Da6a701c568545dCfcB03FcB875f56beddC4to: OrderEvent.getBalance()data: 0x120...65fe0
from	0x5B38Da6a701c568545dCfcB03FcB875f56beddC4
to	OrderEvent.getBalance() 0xcD6a42782d230D7c13A74ddec5dD140e55499Df9
execution cost	23551 gas (Cost only applies when called by a contract)
input	0x120...65fe0
decoded input	{}
decoded output	{
	"0": "uint256: 300"
}
logs	[]
```

## (2) 파일로 컴파일 하세요.
### 컴파일

In [40]:
!solc-windows.exe --optimize --combined-json abi,bin src/OrderEvent.sol > src/OrderEvent.json

## (3) ganache에서 배포하세요.
### 배포

In [41]:
%%writefile src/OrderEventDeploy.js
var Web3 = require('web3');
var _abiBinJson = require('./OrderEvent.json');      //importing a javascript file

var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

contractName=Object.keys(_abiBinJson.contracts); // reading ['src/OrderEvent.sol:OrderEvent']
console.log("- contract name: ", contractName);
_abiArray=_abiBinJson.contracts[contractName].abi;
//_abiArray=JSON.parse(JSON.stringify(_abi));
//_abiArray=JSON.parse(_abi);      //JSON parsing needed!! //SyntaxError: Unexpected token o in JSON at position 1
_bin=_abiBinJson.contracts[contractName].bin;

//console.log("- ABI: " + _abiArray);
//console.log("- Bytecode: " + _bin);

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: "0x"+_bin})
        .send({from: accounts[0], gas: 259210}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
        //.then(function(newContractInstance){
        //    console.log(newContractInstance)
        //});
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()

Overwriting src/OrderEventDeploy.js


In [42]:
!node src/OrderEventDeploy.js

- contract name:  [ 'src/OrderEvent.sol:OrderEvent' ]
Deploying the contract from 0x9A06797e60DbaDe0B84Eec669a7e59541F1586A6
hash: 0xb733635b01ad29057765fad8e03262ab29a7b4cf1abe091e23490e3963b7dfa5
---> The contract deployed to: 0x7BB6d568Fe28AF3f0b509794b3BcC1663F2b0CC3


## (4) 주문을 3건만 하고, 이벤트를 통해 다음과 같이 출력하세요.
항목은 차례대로 주소, 상품항목번호, 주문개수, 단가, 배송지.
```
전송자주소, 1111, 3, 100, 20 2-gil Hongji-dong Jongro-gu Seoul
전송자주소, 1111, 5, 100, 20 2-gil Hongji-dong Jongro-gu Seoul
전송자주소, 1111, 20, 100, 20 2-gil Hongji-dong Jongro-gu Seoul
```
### 사용

In [43]:
%%writefile src/OrderEventUse.js
var Web3=require('web3');
var fs = require('fs');
var _abiBinJson = require('./OrderEvent.json');      //importing a javascript file

//var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
//var web3 = new Web3(new Web3.providers.WebsocketProvider("http://localhost:8345"));
var web3 = new Web3(new Web3.providers.WebsocketProvider("ws://localhost:8345"));
contractName=Object.keys(_abiBinJson.contracts); // reading ['src/OrderEvent.sol:OrderEvent']
//console.log("- contract name: ", contractName); //or console.log(contractName[0]);
_abiArray=_abiBinJson.contracts[contractName].abi; //use just as read from file
//_abiArray=JSON.parse(JSON.stringify(_abi));
//_abiArray=JSON.parse(_abi);      //JSON parsing needed!!
//_bin=_abiBinJson.contracts[contractName].bin;
console.log("- ABI: " + _abiArray);
//console.log("- Bytecode: " + _bin);
//var _test = new web3.eth.Contract(_abiArray,"0x8911bA097c812Bf0B3ff22F90eaf2A905112C5a6");
//var event = _test.events.MyLog({fromBlock: 0}, function (error, result) {
//    if (!error) {
//        console.log("Event fired: " + JSON.stringify(result.returnValues));
//    }
//});

async function doIt() {
    var _order = new web3.eth.Contract(_abiArray, '0x7BB6d568Fe28AF3f0b509794b3BcC1663F2b0CC3');
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    var event = _order.events.OrderLog({
            //filter: {_from: accounts[0], _value: [20,50]},
            //filter: {_from: accounts[0], _value: 30},
            fromBlock: 'latest', toBlock: 'pending'
        }, function (error, result) {
        if (!error) {
            log = JSON.stringify(result.returnValues);
            //console.log("Event fired: " + JSON.stringify(result) + "\n---> " + JSON.stringify(result.returnValues));
            console.log("Event fired: " + log);
            fs.appendFile("src/OrderEvent.txt", log + "\n", "utf-8", function(e) {
                if(!e) {
                    console.log(">> Writing to file");
                }
            });
            //process.exit(1);
        }
    });
    var value;
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    // this will fire an event
    my = await _order.methods.order("0x1111", 3, 100, "20 2-gil Hongji-dong Jongro-gu Seoul")
        .send({from: accounts[0], gas: 100000, value:300});
    // this will fire another event
    my = await _order.methods.order("0x1111", 5, 100, "20 2-gil Hongji-dong Jongro-gu Seoul")
        .send({from: accounts[0], gas: 100000, value:500});
    // this will NOT fire another event
    my = await _order.methods.order("0x1111", 20, 100, "20 2-gil Hongji-dong Jongro-gu Seoul")
        .send({from: accounts[0], gas: 100000, value:2000});
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));    
    process.exit(1); //force exit to disconnect websocket
}

doIt()

Overwriting src/OrderEventUse.js


In [44]:
!node src/OrderEventUse.js

- ABI: [object Object],[object Object],[object Object],[object Object]
Account: 0x9A06797e60DbaDe0B84Eec669a7e59541F1586A6
Balance before: 999999545628000000000
Event fired: {"0":"0x9A06797e60DbaDe0B84Eec669a7e59541F1586A6","1":"0x1111","2":"3","3":"100","4":"20 2-gil Hongji-dong Jongro-gu Seoul","_from":"0x9A06797e60DbaDe0B84Eec669a7e59541F1586A6","_itemId":"0x1111","_quantity":"3","_price":"100","_shipAddress":"20 2-gil Hongji-dong Jongro-gu Seoul"}
>> Writing to file
Event fired: {"0":"0x9A06797e60DbaDe0B84Eec669a7e59541F1586A6","1":"0x1111","2":"5","3":"100","4":"20 2-gil Hongji-dong Jongro-gu Seoul","_from":"0x9A06797e60DbaDe0B84Eec669a7e59541F1586A6","_itemId":"0x1111","_quantity":"5","_price":"100","_shipAddress":"20 2-gil Hongji-dong Jongro-gu Seoul"}
>> Writing to file
Event fired: {"0":"0x9A06797e60DbaDe0B84Eec669a7e59541F1586A6","1":"0x1111","2":"20","3":"100","4":"20 2-gil Hongji-dong Jongro-gu Seoul","_from":"0x9A06797e60DbaDe0B84Eec669a7e59541F1586A6","_itemId":"0x1111","

## (5) 파일에 저장된 파일내역을 출력하세요 (주피터 노트북에서 !type src/OrderEvent.txt)

In [45]:
!type src\OrderEvent.txt

{"0":"0x9A06797e60DbaDe0B84Eec669a7e59541F1586A6","1":"0x1111","2":"3","3":"100","4":"20 2-gil Hongji-dong Jongro-gu Seoul","_from":"0x9A06797e60DbaDe0B84Eec669a7e59541F1586A6","_itemId":"0x1111","_quantity":"3","_price":"100","_shipAddress":"20 2-gil Hongji-dong Jongro-gu Seoul"}
{"0":"0x9A06797e60DbaDe0B84Eec669a7e59541F1586A6","1":"0x1111","2":"5","3":"100","4":"20 2-gil Hongji-dong Jongro-gu Seoul","_from":"0x9A06797e60DbaDe0B84Eec669a7e59541F1586A6","_itemId":"0x1111","_quantity":"5","_price":"100","_shipAddress":"20 2-gil Hongji-dong Jongro-gu Seoul"}
{"0":"0x9A06797e60DbaDe0B84Eec669a7e59541F1586A6","1":"0x1111","2":"20","3":"100","4":"20 2-gil Hongji-dong Jongro-gu Seoul","_from":"0x9A06797e60DbaDe0B84Eec669a7e59541F1586A6","_itemId":"0x1111","_quantity":"20","_price":"100","_shipAddress":"20 2-gil Hongji-dong Jongro-gu Seoul"}
